In [1]:
from utils.url_scraper import extract_content_and_metadata, DATE_FORMATS, DATE_PATTERNS

x = extract_content_and_metadata("https://docs.wire.com/how-to/install/infrastructure-configuration.html#i-have-a-team-larger-than-500-users", DATE_FORMATS, DATE_PATTERNS)

In [4]:
for i in x[:1]:
    print(f"{i['metadata']['headline']}\n{i['content']}")
    
print(f"\n{x[0]['content']}")

Infrastructure configuration options
 This contains instructions to configure specific aspects of your production setup depending on your needs. Depending on your use-case and requirements, you may need to
configure none, or only a subset of the following sections.

 This contains instructions to configure specific aspects of your production setup depending on your needs. Depending on your use-case and requirements, you may need to
configure none, or only a subset of the following sections.


In [35]:
import aiohttp
import asyncio
from urllib.parse import urljoin, urlparse
import re
from datetime import datetime
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import requests
from collections import defaultdict

def requests_retry_session(
        retries=3,
        backoff_factor=0.3,
        status_forcelist=(500, 502, 504),
        session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

my_url = "https://docs.wire.com/how-to/install/infrastructure-configuration.html#i-have-a-team-larger-than-500-users"
session = requests_retry_session()
response = session.get(my_url, timeout=20)

# Check if request was successful
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

content_by_headline = defaultdict(str)
current_header = None
for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre']):
    if element.name.startswith('h'):
        # Headline
        print(f"$$$ {element.name} $$$ {element.get_text(strip=False)}")
        current_header = element.get_text(strip=True)
        current_header = current_header.replace("\n", "")
        current_header = re.sub(r"\s+", " ", current_header)
        current_header = re.sub(r'[^\x00-\x7F]+', '', current_header)

    elif element.name in ['p', 'pre'] and current_header:
        # Append the text under the last seen headline
        print(f"^^^ {element.name} ^^^ {element.get_text(strip=False)}")
        if element.name == 'pre':
            new_content = element.get_text(strip=False)
        else:
            new_content = re.sub(r'\s+', ' ', element.get_text(strip=False)).replace("\n", " ")
        content_by_headline[current_header] += f"\n{new_content}"
            

$$$ h1 $$$ Infrastructure configuration options
^^^ p ^^^ This contains instructions to configure specific aspects of your production setup depending on your needs.
^^^ p ^^^ Depending on your use-case and requirements, you may need to
configure none, or only a subset of the following sections.
$$$ h2 $$$ Redirect some traffic through a http(s) proxy
^^^ p ^^^ In case you wish to use http(s) proxies, you can add a configuration like this to the wire-server services in question:
^^^ p ^^^ Assuming your proxy can be reached from within Kubernetes at http://proxy:8080, add the following for each affected service (e.g. gundeck) to your Helm overrides in values/wire-server/values.yaml :
^^^ pre ^^^ gundeck:
  # ...
  config:
    # ...
    proxy:
      httpProxy: "http://proxy:8080"
      httpsProxy: "http://proxy:8080"
      noProxyList:
        - "localhost"
        - "127.0.0.1"
        - "10.0.0.0/8"
        - "elasticsearch-external"
        - "cassandra-external"
        - "redis-eph

In [36]:
print(content_by_headline['Redirect some traffic through a http(s) proxy'])


In case you wish to use http(s) proxies, you can add a configuration like this to the wire-server services in question:
Assuming your proxy can be reached from within Kubernetes at http://proxy:8080, add the following for each affected service (e.g. gundeck) to your Helm overrides in values/wire-server/values.yaml :
gundeck:
  # ...
  config:
    # ...
    proxy:
      httpProxy: "http://proxy:8080"
      httpsProxy: "http://proxy:8080"
      noProxyList:
        - "localhost"
        - "127.0.0.1"
        - "10.0.0.0/8"
        - "elasticsearch-external"
        - "cassandra-external"
        - "redis-ephemeral"
        - "fake-aws-sqs"
        - "fake-aws-dynamodb"
        - "fake-aws-sns"
        - "brig"
        - "cargohold"
        - "galley"
        - "gundeck"
        - "proxy"
        - "spar"
        - "federator"
        - "cannon"
        - "cannon-0.cannon.default"
        - "cannon-1.cannon.default"
        - "cannon-2.cannon.default"

Depending on your setup, you may ne

In [38]:
for key, content in content_by_headline.items():
    print(f"{key}{content}\n\n\n")

Infrastructure configuration options
This contains instructions to configure specific aspects of your production setup depending on your needs.
Depending on your use-case and requirements, you may need to configure none, or only a subset of the following sections.



Redirect some traffic through a http(s) proxy
In case you wish to use http(s) proxies, you can add a configuration like this to the wire-server services in question:
Assuming your proxy can be reached from within Kubernetes at http://proxy:8080, add the following for each affected service (e.g. gundeck) to your Helm overrides in values/wire-server/values.yaml :
gundeck:
  # ...
  config:
    # ...
    proxy:
      httpProxy: "http://proxy:8080"
      httpsProxy: "http://proxy:8080"
      noProxyList:
        - "localhost"
        - "127.0.0.1"
        - "10.0.0.0/8"
        - "elasticsearch-external"
        - "cassandra-external"
        - "redis-ephemeral"
        - "fake-aws-sqs"
        - "fake-aws-dynamodb"
        - 